In [4]:
import os
%pwd

'c:\\Users\\vaish\\Documents\\Ravi\\chicken_disease_classification\\Chicken-Disease-Classification-Project'

In [3]:
os.chdir("../")
%pwd

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [21]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath= CONFIG_FILE_PATH,
                 parmas_filepath= PARAMS_PATH_FILE):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(parmas_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [23]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import  logger
from cnnClassifier.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    
    

In [27]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-11 01:48:39,047: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-11 01:48:39,049: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-11 01:48:39,052: INFO: common: created directory at: artifacts]
[2023-06-11 01:48:39,054: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-11 01:48:40,833: INFO: 3912014281: artifacts/data_ingestion/data.zip download! with following info: 
Server: GitHub.com
Date: Sat, 10 Jun 2023 20:18:38 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"d9962d00bbd18937e837d9816fea44b8"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; block-all-mixe

BadZipFile: File is not a zip file